In [1]:
import pickle

from dataset import NewsDataset
from BertModel import WeightedBertForSequenceClassification

from smooth_gradient import SmoothGradient
from integrated_gradient import IntegratedGradient

import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer
import pandas as pd
from IPython.display import display, HTML
import numpy as np
from transformers import BertConfig, BertTokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
train_df = pd.read_csv('../tcav2/data/los_nicu_train_admissions.csv')
val_df = pd.read_csv('../tcav2/data/los_nicu_val_admissions.csv')
test_df = pd.read_csv('../tcav2/data/los_nicu_test_admissions.csv')

In [6]:
train_df

,Unnamed: 0,new_HADM,Unnamed: 0.1,TEXT,label,ROW_ID,SUBJECT_ID,CHARTTIME,AGE,GENDER
0,0,104692,403165,Nsg admit/transfer note\n [**Hospital 20**] ...,0,1711540,2437,2100-08-03 11:13:00,0.346528,M
1,1,108473,410336,NURSING NOTE\nBaby Girl [**Known lastname 1591...,0,2028728,25831,2100-08-25 11:18:00,0.393056,F
2,2,135605,173472,Neonatology Attending\n\n1790 gram 34 [**4-14*...,1,2072280,31329,2100-08-29 23:17:00,0.950000,F
3,3,111007,402527,Neonatology Attending Triage Note\n\nAsked by ...,0,1689273,783,2100-09-09 14:27:00,0.477083,F
4,4,119916,366566,Neonatology\nPatient is 2085 gram product of 3...,0,1763950,6122,2100-09-17 17:14:00,0.621528,M
...,...,...,...,...,...,...,...,...,...,...
3775,3775,187343,408082,Admission Note\nBaby brought to NICU from L&D ...,0,1961023,20650,2201-05-15 15:18:00,0.981250,M
3776,3776,161900,402464,Admission Note\nReceived baby from L&D for sep...,0,1698622,1424,2201-05-21 12:17:00,0.356944,M
3777,3777,189638,252703,Neonatology Attending Admit Note:\n\n33 [**7-1...,0,1819238,10100,2201-05-21 14:53:00,0.588889,M
3778,3778,122461,400567,Admission Note\nNursing Admit and transfer not...,0,1931947,18079,2201-06-07 16:57:00,0.606250,M


In [7]:
classes = list(test_df.columns[3:])

In [8]:
import pandas as pd

# grads_df = pd.read_csv('all_test_grads2.csv')
grads_df = pd.read_csv('all_nicu_grads.csv')

grads_df['tokens']=grads_df['tokens'].apply(eval)

grads_df['grad'] = grads_df['grad'].apply(eval)

In [9]:
# grads_df=grads_df[-1*len(test_df):]

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
model_path = '../torch_tcav/model_train_scripts/checkpoints/FINALWeightedBlueBert512NicuLosAdmissions_2/checkpoint-26015'
bert = WeightedBertForSequenceClassification.from_pretrained(model_path,output_attentions=True)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert.eval();
tokens = tokenizer.encode(['This is a test','so is this'], padding='max_length', max_length=512)
layer_acts = bert(input_ids=torch.LongTensor([tokens,tokens]))


In [13]:
layer_acts

tensor([[-2.2032,  1.8657],
        [-2.2032,  1.8657]], grad_fn=<AddmmBackward>)

In [14]:
quotes_list = ['IUI, trichorionic, triamniotic','Antepartum remarkable for ART conception with subsequent twinning of one embryo',
              'Growth better than siblings, but still only around 10-25%','Infant TRIPLET BORN AT 31 WEEKS GESTATION']

In [15]:
grads_df['ROW_ID'] = list(train_df['ROW_ID'])+list(val_df['ROW_ID'])+list(test_df['ROW_ID'])
grads_df['TEXT'] = list(train_df['TEXT'])+list(val_df['TEXT'])+list(test_df['TEXT'])

In [16]:
# list(grads_df[grads_df['TEXT'].apply(lambda x: 'eclampsia' in x.lower())]['TEXT'])

In [17]:
note='Past Medical History:- HTN- HL- arrhythmia-  on stress testing [**3-/2145**], increased ectopy including frequent APB\'s, couplets, triplets and several runs of nonsustained wide complex tachycardia.'

In [18]:
texts=['trhiu=','']
config = BertConfig()
batch_size=1
all_instances = []
for text in texts:
    test_example = [[note],['this also is']]

    test_dataset = NewsDataset(
        data_list=test_example,
        tokenizer=tokenizer,
        max_length=config.max_position_embeddings, 
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
    )

    integrated_grad = IntegratedGradient(
        bert, 
        criterion, 
        tokenizer, 
        show_progress=True,
        encoder="bert"
    )
    instances = integrated_grad.saliency_interpret(test_dataloader)
    all_instances.append(instances)

  0%|          | 0/1 [00:00<?, ?it/s]/gpfs/gsfs12/users/moldwinar/environments/torchtcav/lib/python3.6/site-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 1/1 [01:03<00:00, 63.41s/it]


In [19]:
# instances

In [20]:
test_df = pd.read_csv('../tcav2/data/los_nicu_test_admissions.csv')
notes_path='../../LHC_mimic/mimic3_1.4/raw_data/NOTEEVENTS.csv'
test_df= test_df.merge(pd.read_csv(notes_path, usecols=['ROW_ID','HADM_ID']), on='ROW_ID',how='left')
dx_df = pd.read_csv('../../LHC_mimic/mimic3_1.4/raw_data/DIAGNOSES_ICD.csv')
dx_df = pd.DataFrame(dx_df.groupby('HADM_ID')['ICD9_CODE'].apply(list)).reset_index()
test_df=test_df.merge(dx_df[['HADM_ID','ICD9_CODE']], on='HADM_ID',how='left')

In [55]:
test_df[test_df['TEXT'].apply(lambda x: 'mottled' in x.lower())][['TEXT','ROW_ID','ICD9_CODE']].to_dict(orient='records')

[{'TEXT': 'Admission Note\nNeonatology H&P\n\nBaby Girl [**Known lastname 2753**] is a newborn 35 wk infant admitted to the NICU with prematurity and respiratory distress.\n\nShe was born at 12:06 am this morning as the 3495 gram product of a 35 week twin gestation pregnancy to a 28 y.o. G2 P1-2 mother with [**Name (NI) 44**] [**2105-8-15**].  Prenatal labs included BT A+/Ab-, RPR NR, RI, HBsAg-, GBS unknown, and HIV negative.  Maternal history notable for HSV, not currently active, and depression, currently treated with prozac.  This pregnancy was notable for spontaneous mono/di twin gestation.  Mother was followed closely at [**Name (NI) 67**] Hospital by her OB and by Dr. [**Last Name (STitle) **] for concerns of twin-twin transfusion syndrome, with up to a 30% growth discrepancy seen.  Amniotic fluid volumes, however, remained similar, and no evidence of hydrops was noted.\n\nMother presented to her OB today with complaints of decreased fetal activity.  Ultrasound showed in-utero f

In [23]:
coloder_string = integrated_grad.colorize(instances[0])
display(HTML(coloder_string))

Using bos_token, but it is not set yet.


In [24]:
texts = train_df[train_df['TEXT'].apply(lambda x: any([y in x for y in quotes_list]))]

grads_df.merge(texts[['TEXT','ROW_ID']], on='ROW_ID',how='inner')

,Unnamed: 0,tokens,grad,label,prob,ROW_ID,TEXT_x,TEXT_y
0,1286,"[[CLS], neon, ##ato, ##logy, attending, admiss...","[3.172817741869949e-05, 0.0012127560330554843,...",0,0.999892,1741459,Neonatology Attending Admission Note\n\nInfnat...,Neonatology Attending Admission Note\n\nInfnat...
1,2713,"[[CLS], neon, ##ato, ##logy, attending, pre, #...","[0.001805896288715303, 0.0003229107824154198, ...",1,0.999999,1924622,Neonatology Attending\n\nPreterm infant with r...,Neonatology Attending\n\nPreterm infant with r...


In [25]:
texts_to_visualize=pd.DataFrame([1995762,960886,1797993,2021380  ,1756099,33367,1891184,1712408,1887933    ], columns=['ROW_ID']).merge(grads_df, how='left',on='ROW_ID')

In [26]:
texts_to_visualize

,ROW_ID,Unnamed: 0,tokens,grad,label,prob,TEXT
0,1995762,NaN,NaN,NaN,NaN,NaN,NaN
1,960886,NaN,NaN,NaN,NaN,NaN,NaN
2,1797993,4531.0,"[[CLS], neon, ##ato, ##logy, attending, pre, #...","[0.0006579961045645177, 0.0005383858806453645,...",1.0,0.999999,Neonatology Attending\n\nPreterm infant with r...
3,2021380,6902.0,"[[CLS], neon, ##ato, ##logy, attending, admiss...","[0.003377385903149843, 0.00105695438105613, 0....",1.0,0.999999,Neonatology Attending Admission Note\n\nInfant...
4,1756099,2097.0,"[[CLS], nic, ##u, attending, admission, note, ...","[0.0004335545818321407, 0.00045383695396594703...",0.0,0.999999,NICU Attending Admission Note\n\nID: 31 week ...
5,33367,NaN,NaN,NaN,NaN,NaN,NaN
6,1891184,766.0,"[[CLS], neon, ##ato, ##logy, attending, admit,...","[0.004383975174278021, 0.0008386531262658536, ...",0.0,0.999999,Neonatology Attending Admit Note:\n\n32 [**6-1...
7,1712408,2425.0,"[[CLS], admission, note, ob, -, [, *, *, docto...","[0.0014790587592869997, 0.004091564565896988, ...",1.0,0.999999,Admission Note\nOb-[**Doctor Last Name 442**]\...
8,1887933,6052.0,"[[CLS], admission, note, neon, ##atal, attendi...","[0.0015696268528699875, 0.0019188850419595838,...",1.0,0.999999,Admission Note\nNeonatal Attending Admission N...


In [27]:
def select_context_idx(row, start, stop):
    row['grad']=[0.03]+row['grad'][start:stop]
    row['tokens']=['']+row['tokens'][start:stop]
    print(max(row['grad']))
    return row

In [28]:
coloder_string = integrated_grad.colorize(select_context_idx(texts_to_visualize.iloc[1],145,170).to_dict())
display(HTML(coloder_string))
print()
coloder_string = integrated_grad.colorize(select_context_idx(texts_to_visualize.iloc[0],341,363).to_dict())
display(HTML(coloder_string))

print()
coloder_string = integrated_grad.colorize(select_context_idx(texts_to_visualize.iloc[2],54,75).to_dict())
display(HTML(coloder_string))
print()
coloder_string = integrated_grad.colorize(select_context_idx(texts_to_visualize.iloc[3],169,188).to_dict())
display(HTML(coloder_string))
print()
coloder_string = integrated_grad.colorize(select_context_idx(texts_to_visualize.iloc[4],203,225).to_dict())
display(HTML(coloder_string))
print()
coloder_string = integrated_grad.colorize(select_context_idx(texts_to_visualize.iloc[5],352,-152).to_dict())
display(HTML(coloder_string))
print()
coloder_string = integrated_grad.colorize(select_context_idx(texts_to_visualize.iloc[6],8,31).to_dict())
display(HTML(coloder_string))
print()
coloder_string = integrated_grad.colorize(select_context_idx(texts_to_visualize.iloc[6],305,338).to_dict())
display(HTML(coloder_string))
print()
coloder_string = integrated_grad.colorize(select_context_idx(texts_to_visualize.iloc[7],269,286).to_dict())
display(HTML(coloder_string))
print()
coloder_string = integrated_grad.colorize(select_context_idx(texts_to_visualize.iloc[8],323,334).to_dict())
display(HTML(coloder_string))


TypeError: 'float' object is not subscriptable

In [30]:
grads_df=grads_df.iloc[-1*len(test_df):]

In [32]:
grads_df['TEXT']=list(test_df['TEXT'])

In [33]:
grads_df['tokens'].apply(lambda x: len(x))

5671    435
5672    369
5673     43
5674    512
5675    399
       ... 
7557    250
7558    349
7559    430
7560    512
7561    159
Name: tokens, Length: 1891, dtype: int64

In [34]:
grads_df['TEXT'].apply(lambda x: 'TRIPLET BORN AT 31 ' in x).value_counts()

False    1891
Name: TEXT, dtype: int64

In [35]:
ex = grads_df[grads_df['TEXT'].apply(lambda x: 'TRIPLET BORN AT 31 ' in x)].iloc[0]
coloder_string=integrated_grad.colorize({'tokens':ex['tokens'], 'grad':ex['grad'], 'label':ex['label'],'prob':ex['prob']})
display(HTML(coloder_string))

IndexError: single positional indexer is out-of-bounds

In [36]:
from torch.nn.functional import softmax
print(grads_df.iloc[0])
torch.max(softmax(bert(input_ids=torch.tensor([tokenizer.encode(test_df['TEXT'].iloc[0])])), dim=-1),dim=-1)

Unnamed: 0                                                 5671
tokens        [[CLS], neon, ##ato, ##logy, attending, tri, #...
grad          [0.0022628488950431347, 0.0002968515909742564,...
label                                                         0
prob                                                   0.999999
ROW_ID                                                  2051008
TEXT          Neonatology Attending Triage Note\n\nAsked by ...
Name: 5671, dtype: object


torch.return_types.max(
values=tensor([1.0000], grad_fn=<MaxBackward0>),
indices=tensor([0]))

In [37]:
from sklearn.metrics import classification_report
classification_report(grads_df['label'], test_df['label'], output_dict=True)

{'0': {'precision': 0.9070100143061517,
  'recall': 0.902491103202847,
  'f1-score': 0.904744916161256,
  'support': 1405},
 '1': {'precision': 0.7221095334685599,
  'recall': 0.7325102880658436,
  'f1-score': 0.7272727272727274,
  'support': 486},
 'accuracy': 0.8588048651507139,
 'macro avg': {'precision': 0.8145597738873558,
  'recall': 0.8175006956343454,
  'f1-score': 0.8160088217169916,
  'support': 1891},
 'weighted avg': {'precision': 0.8594893196011969,
  'recall': 0.8588048651507139,
  'f1-score': 0.8591333435542623,
  'support': 1891}}

In [38]:
def unravel(lst):
    return [i for j in lst for i in j]

In [39]:
# label = 1
all_class_grads = pd.DataFrame(  list(zip( unravel(grads_df['tokens']),  unravel(grads_df['grad']) )) , columns=['tokens', 'grad']  )

In [40]:
word =''
score=0
words=[]
scores=[]
for i in reversed(range(len(all_class_grads))):
    token=all_class_grads['tokens'].iloc[i]
    if token.startswith('##'):
        word = token[2:]+word
        score = max(score,all_class_grads['grad'].iloc[i] )
#         print(word)
    else:
        word = token+word
#         print(word)
        words.append(word)
        score = max(score,all_class_grads['grad'].iloc[i] )
        scores.append(score)
        word=''
        score=0

In [41]:
tokens_grouped = pd.DataFrame(list(zip(words, scores)), columns=['words', 'grad']).groupby('words')
means = tokens_grouped['grad'].mean()
cts = tokens_grouped['grad'].count()
wrds = tokens_grouped.apply(lambda group: str(group.name))
class_df = pd.DataFrame(list(zip(wrds,means,cts)), columns=['word','grad_mean','word_count' ]).sort_values(by='grad_mean',ascending=False)
class_df = class_df[class_df['word_count']>5]
print(len(class_df))
# print(class_df.head(25).to_latex(index=False, float_format="%.3f"))
class_df.head(27)

3074


,word,grad_mean,word_count
10417,sheepskin,0.047350,30
8196,mottled,0.033975,9
7637,laser,0.023670,6
8550,notified,0.022094,16
3980,arrived,0.021549,81
10713,starter,0.021483,18
11809,vss,0.021290,150
7518,iugr,0.019502,65
4180,babycares,0.017878,7
11345,transfer,0.017731,431


In [51]:
class_df[class_df['word'].apply(lambda x: x.startswith('['))]

,word,grad_mean,word_count
3422,[SEP],0.015766,1891
3421,[CLS],0.001626,1891
3420,[,0.001559,7359


In [42]:
word =''
score=0
negative_words=[]
negative_scores=[]
for i in reversed(range(len(all_negative_grads))):
    token=all_negative_grads['tokens'].iloc[i]
    if token.startswith('##'):
        word = token[2:]+word
        score = max(score,all_negative_grads['grad'].iloc[i] )
#         print(word)
    else:
        word = token+word
#         print(word)
        negative_words.append(word)
        score = max(score,all_negative_grads['grad'].iloc[i] )
        negative_scores.append(score)
        word=''
        score=0

NameError: name 'all_negative_grads' is not defined

In [ ]:
tokens_grouped = pd.DataFrame(list(zip(negative_words, negative_scores)), columns=['words', 'grad']).groupby('words')
means = tokens_grouped['grad'].mean()
cts = tokens_grouped['grad'].count()
wrds = tokens_grouped.apply(lambda group: str(group.name))
negative_df = pd.DataFrame(list(zip(wrds,means,cts)), columns=['word','grad_mean','word_count' ]).sort_values(by='grad_mean',ascending=False)
negative_df = negative_df[negative_df['word_count']>5]
print(negative_df.head(25).to_latex(index=False, float_format="%.3f"))

In [ ]:
list(means

In [ ]:
pd.DataFrame(list(zip(words, scores)), columns=['words', 'grad'])

In [57]:
import pandas as pd
pd.DataFrame(list(zip(instances[0]['tokens'], instances[0]['grad'])))

IndexError: list index out of range

In [20]:
instances

[{'tokens': ['[CLS]',
   "'",
   'patient',
   'was',
   'a',
   'pleasant',
   'male',
   'in',
   'his',
   'early',
   '40',
   '##s',
   'who',
   'was',
   'given',
   'pl',
   '##avi',
   '##x',
   'and',
   'as',
   '##pi',
   '##rin',
   'for',
   'cad',
   'and',
   'required',
   'cab',
   '##g',
   "'",
   '[SEP]',
   'transform',
   '##er',
   '-',
   'based',
   'models',
   'have',
   'taken',
   'a',
   'leading',
   'role',
   'in',
   'nl',
   '##p',
   'today',
   '.',
   '[SEP]'],
  'grad': [0.0024040760472416878,
   0.011202486231923103,
   0.013759119436144829,
   0.002332319738343358,
   0.011373970657587051,
   0.02735648676753044,
   0.015535157173871994,
   0.030777039006352425,
   0.008332711644470692,
   0.013340604491531849,
   0.0001066805562004447,
   0.0027349493466317654,
   0.004292184952646494,
   0.00442542415112257,
   0.007150531280785799,
   0.0028611449524760246,
   0.015010221861302853,
   0.018782952800393105,
   0.01669878326356411,
   0.009789